# Pipeline ETL com Python: Análise de Risco de Diabetes

## Descrição do Projeto

Este notebook apresenta a construção de um pipeline ETL (Extract, Transform, Load) prático e voltado para a área da saúde.

O objetivo é simular o processamento de dados clínicos para rastreamento e identificação de riscos de Diabetes Mellitus. Utilizaremos um conjunto de dados sintético (fictício) que contém informações antropométricas e laboratoriais, como IMC e Hemoglobina Glicada (HbA1c).

### **🎯 Objetivos Específicos**
- **Extração:** Coletar dados de um repositório público (CSV).

- **Transformação:**
  *  Limpar e preparar os dados.
  *  Aplicar critérios clínicos reais (Baseados na Associação Americana de Diabetes) para classificar pacientes em Normal, Pré-Diabetes e Diabetes.
  *  Analisar quais fatores (como IMC e Idade) estão mais associados ao aumento da HbA1c.

- **Carga:** Salvar os dados processados e enriquecidos prontos para análise ou uso em modelos preditivos.

### 🛠️ Ferramentas Utilizadas
- **Python:** Linguagem de programação.

- **Pandas:** Manipulação e análise de dados tabulares.

- **Scikit-Learn:** (Opcional) Para modelagem preditiva simples.

Scikit-learn foi importado para fazer a parte de ML. O Scikit-Learn é uma biblioteca grande, então geralmente importamos apenas as partes específicas que vamos usar (modelos, métricas, funções de divisão de dados).

In [8]:
# Importanto Bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Extração
Baixando Dataset direto da biblioteca do Kaggle


In [9]:
# 1. Instalando e importando a biblioteca do Kaggle (caso não tenha)
import kagglehub
import os

# 2. Baixando o dataset automaticamente
# Isso vai baixar e salvar numa pasta temporária do Colab
path = kagglehub.dataset_download("abdelrahmanzayed2/diabetes-prediction-dataset-csv-zip")

print("📂 Pasta onde o arquivo foi salvo:", path)

# 3. Encontrando o arquivo CSV dentro da pasta baixada
# Como o download pode vir numa pasta com vários arquivos, vamos pegar o primeiro CSV que encontrarmos
arquivos = os.listdir(path)
csv_file = [f for f in arquivos if f.endswith('.csv')][0] # Pega o primeiro arquivo que termina com .csv
caminho_completo = os.path.join(path, csv_file)

print(f"📄 Lendo o arquivo: {csv_file}")

# 4. Lendo com Pandas
df = pd.read_csv(caminho_completo)

# 5. Visualizando
print("✅ Dados carregados com sucesso via KaggleHub!")
display(df.head())


📂 Pasta onde o arquivo foi salvo: /root/.cache/kagglehub/datasets/abdelrahmanzayed2/diabetes-prediction-dataset-csv-zip/versions/1
📄 Lendo o arquivo: diabetes_prediction_dataset.csv
✅ Dados carregados com sucesso via KaggleHub!


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


# Transform (Transformação e Limpeza)
Agora que temos a tabela df carregada, precisamos prepará-la. Olhando para as colunas (gender, age, hypertension, heart_disease, bmi, HbA1c_level, blood_glucose_level, diabetes), vamos aplicar seu conhecimento clínico.

##Nossos objetivos nesta etapa:

- **Criar a Classificação Clínica (HbA1c):** O dataset tem o nível numérico da HbA1c, mas clinicamente queremos saber quem é "Normal", "Pré-Diabético" ou "Diabético". Vamos criar essa coluna nova.

- **Transformar Textos em Números:** Modelos de IA (como a Árvore de Decisão que vamos usar depois) não entendem textos como "Female" ou "Male", nem "No Info". Precisamos converter isso para números.

### Criar a Coluna de Classificação de Risco
Cria uma função simples que aplica as regras da American Diabetes Association e classifica de acordo com o valor da HbA1c se o indivíduo tem Diabetes, Pré-Diabetes ou Nao possui a doenca.

In [10]:
# --- ETAPA DE TRANSFORMAÇÃO (TRANSFORM) ---

# 1. Função para classificar o risco com base na HbA1c
def classificar_risco_hba1c(hba1c):
    if hba1c >= 6.5:
        return 'Diabetes'
    elif hba1c >= 5.7:
        return 'Pre-Diabetes'
    else:
        return 'Normal'

# Aplicando a função em cada linha da tabela para criar uma nova coluna
df['classificacao_risco'] = df['HbA1c_level'].apply(classificar_risco_hba1c)

# Vamos ver como ficou a distribuição? (Quantos de cada tipo)
print("Distribuição dos Pacientes por Categoria de Risco (Baseado na HbA1c):")
print(df['classificacao_risco'].value_counts())

# Mostrando a tabela com a nova coluna
display(df[['HbA1c_level', 'classificacao_risco']].head())


Distribuição dos Pacientes por Categoria de Risco (Baseado na HbA1c):
classificacao_risco
Pre-Diabetes    41346
Normal          37857
Diabetes        20797
Name: count, dtype: int64


,HbA1c_level,classificacao_risco
0,6.6,Diabetes
1,6.6,Diabetes
2,5.7,Pre-Diabetes
3,5.0,Normal
4,4.8,Normal


## Transformar Variáveis de Texto (Categorias) em Números

A coluna gender ou smoking_history, elas têm texto. Para fazermos O modelo matemático precisa que isso vire número.

Exemplo: Female = 0, Male = 1

Exemplo: never = 0, current = 1, etc.

Há uma ferramenta do Pandas chamada **factorize** ou **map** que faz isso de forma simples.

In [11]:
# 2. Convertendo variáveis de texto para números (Encoding)

# Gênero
# Verificar quais gêneros existem primeiro
print("Gêneros encontrados:", df['gender'].unique())
# Transformando: Female=0, Male=1, Other=2 (se houver)
df['gender_code'] = df['gender'].astype('category').cat.codes

# Histórico de Fumo
print("Histórico de Fumo:", df['smoking_history'].unique())
# Transformando cada categoria de fumo em um número sequencial
df['smoking_code'] = df['smoking_history'].astype('category').cat.codes

# Código numérico para o nosso ALVO (Risco)
# Normal=0, Pre-Diabetes=1, Diabetes=2
# Para garantir a ordem clínica (Normal < Pre < Diabetes), vamos usar map manual:
mapa_risco = {'Normal': 0, 'Pre-Diabetes': 1, 'Diabetes': 2}
df['risco_code'] = df['classificacao_risco'].map(mapa_risco)

print("\n Transformação concluída! Colunas numéricas criadas.")
display(df[['gender', 'gender_code', 'smoking_history', 'smoking_code', 'classificacao_risco', 'risco_code']].head())


Gêneros encontrados: ['Female' 'Male' 'Other']
Histórico de Fumo: ['never' 'No Info' 'current' 'former' 'ever' 'not current']

 Transformação concluída! Colunas numéricas criadas.


,gender,gender_code,smoking_history,smoking_code,classificacao_risco,risco_code
0,Female,0,never,4,Diabetes,2
1,Female,0,No Info,0,Diabetes,2
2,Male,1,never,4,Pre-Diabetes,1
3,Female,0,current,1,Normal,0
4,Male,1,current,1,Normal,0


## Modelagem

Transformando os dados brutos em uma predição (uma nova informação que não existia antes).

Vamos usar um algoritmo chamado **Árvore de Decisão (Decision Tree)**.

Uma árvore de decisão é um modelo que toma decisões fazendo perguntas sequenciais sobre as variáveis até chegar a uma conclusão (classe) em uma "folha" da árvore. Ela é muito usada em saúde porque o raciocínio se parece com um raciocínio clínico ou um fluxograma de triagem.

**Ex:**
- "A glicemia está alta?" -> Se sim, vai para um lado.
- "O IMC é maior que 30?" -> Se sim, vai para outro.

### Estrutura básica

A árvore tem três tipos principais de elementos.

**Nó raiz:** é o primeiro ponto de decisão, a variável mais importante (por exemplo, "HbA1c ≥ 6,5?" ou "IMC ≥ 30?").

**Nós internos:** cada nó faz um teste em uma variável (por exemplo, "idade ≥ 45?") e divide os pacientes em grupos diferentes.

**Folhas:** são os nós finais, onde o modelo dá a resposta (por exemplo, "Diabetes", "Pré-diabetes", "Normal").

### Como a árvore "aprende"

No treino, o algoritmo recebe muitos exemplos com entrada (idade, IMC, hipertensão, etc.) e saída conhecida (risco de diabetes).  
Ele testa várias formas de dividir os dados em cada nó e escolhe a divisão que melhor separa as classes, medindo essa "qualidade" com critérios como entropia, ganho de informação ou índice Gini.

Em cada etapa, o objetivo é deixar os grupos o mais "puros" possível, ou seja, com o máximo de pacientes da mesma classe (por exemplo, quase só diabéticos em um lado e quase só não diabéticos no outro).

Esse processo se repete recursivamente: cada novo nó tenta separar ainda mais até chegar às folhas.

### Decisão na prática (fase de uso)

Depois de treinada, usar a árvore é como seguir um fluxograma.

O paciente entra na raiz, o modelo verifica a condição do nó (por exemplo, "IMC ≥ 30?") e segue o ramo "sim" ou "não".

Em cada passo, outra condição é checada (idade, hipertensão, tabagismo, etc.), até chegar a uma folha com a classe prevista.

Essa transparência é uma das maiores vantagens: é possível justificar decisões ("o modelo classificou como alto risco porque IMC ≥ X, idade ≥ Y e hipertensão = sim").

### Vantagens e limitações

**Principais vantagens:** a árvore é fácil de entender, não exige padronizar dados e lida tanto com variáveis numéricas quanto categóricas.  
**Principais limitações:** se deixar crescer demais (muitos níveis), ela pode decorar o conjunto de treino (overfitting), por isso se usa poda, profundidade máxima e número mínimo de exemplos por nó.

Neste projeto, estou usando uma árvore de classificação deste modo ela aprende a mapear características como idade, IMC, hipertensão, tabagismo e sexo para classes de risco (Normal, Pré-DM, DM) de maneira legível e próxima do raciocínio clínico.


Vamos dividir esse processo em 3 partes pequenas na mesma célula:

- Separar as variáveis: Definir o que é PERGUNTA (ex: idade, IMC, histórico) e o que é RESPOSTA (o risco de diabetes).

- Dividir os dados: Separar uma parte para a IA "estudar" (Treino) e uma parte para a IA "fazer a prova" (Teste).

- Treinar: Ensinar o modelo.

In [12]:
# --- ETAPA DE MODELAGEM ---

# 1. Definindo as variáveis
# X = Dados que usamos para prever (Características/Features)
# ATENÇÃO: Removemos 'HbA1c_level', 'blood_glucose_level' e 'diabetes' original
# Motivo: Se deixarmos a HbA1c ou Glicose, o modelo "descobre" a resposta fácil demais.
# Queremos ver se IDADE, IMC e HIPERTENSÃO conseguem prever o risco sozinhas.
colunas_preditoras = ['gender_code', 'age', 'hypertension', 'heart_disease', 'smoking_code', 'bmi']
X = df[colunas_preditoras]

# Y = A resposta que queremos alcançar (Risco: 0, 1 ou 2)
y = df['risco_code']

# 2. Dividindo em Treino (70%) e Teste (30%)
# random_state=42 serve para o resultado ser sempre igual quando rodarmos de novo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Tamanho do Treino: {X_train.shape[0]} pacientes")
print(f"Tamanho do Teste: {X_test.shape[0]} pacientes")

# 3. Treinando o Modelo (Decision Tree)
# max_depth=4 limita a árvore a 4 perguntas de profundidade, para ficar fácil de entender
modelo = DecisionTreeClassifier(max_depth=4, random_state=42)
modelo.fit(X_train, y_train)

print("\n🤖 Modelo treinado com sucesso!")


Tamanho do Treino: 70000 pacientes
Tamanho do Teste: 30000 pacientes

🤖 Modelo treinado com sucesso!


### Avaliando o Modelo
Agora vamos usar os dados de teste para ver se ele consegue prever corretamente.

In [14]:
from sklearn.metrics import classification_report

# --- AVALIANDO O MODELO ---

# 1. Fazendo previsões nos dados de TESTE (que o modelo nunca viu)
previsoes_teste = modelo.predict(X_test)

# 2. Calculando a acurácia (percentual de acertos)
acuracia = accuracy_score(y_test, previsoes_teste)
print(f"🎯 **ACURÁCIA DO MODELO: {acuracia:.2%}**")
print(f"Isso significa que o modelo acertou {acuracia*100:.0f}% dos casos!")

# 3. Matriz de Confusão (mostra onde ele errou e acertou)
print("\n📊 **MATRIZ DE CONFUSÃO** (Linhas=Real, Colunas=Predito):")
print("          Normal    Pré-DM     DM")
print(confusion_matrix(y_test, previsoes_teste))

# 4. Relatório detalhado
print("\n📋 **RELATÓRIO DETALHADO POR CLASSE:**")
print(classification_report(y_test, previsoes_teste,
                          target_names=['Normal', 'Pré-Diabetes', 'Diabetes']))

# 5. Qual variável foi mais importante? (O que a árvore "achou" mais relevante)
importancias = pd.DataFrame({
    'Variavel': colunas_preditoras,
    'Importancia': modelo.feature_importances_
}).sort_values('Importancia', ascending=False)

print("\n🏆 **IMPORTÂNCIA DAS VARIÁVEIS** (O que define o risco?):")
display(importancias)

🎯 **ACURÁCIA DO MODELO: 41.14%**
Isso significa que o modelo acertou 41% dos casos!

📊 **MATRIZ DE CONFUSÃO** (Linhas=Real, Colunas=Predito):
          Normal    Pré-DM     DM
[[  477 10766    53]
 [  533 11777    98]
 [  229  5978    89]]

📋 **RELATÓRIO DETALHADO POR CLASSE:**
              precision    recall  f1-score   support

      Normal       0.38      0.04      0.08     11296
Pré-Diabetes       0.41      0.95      0.58     12408
    Diabetes       0.37      0.01      0.03      6296

    accuracy                           0.41     30000
   macro avg       0.39      0.34      0.23     30000
weighted avg       0.39      0.41      0.27     30000


🏆 **IMPORTÂNCIA DAS VARIÁVEIS** (O que define o risco?):


,Variavel,Importancia
1,age,0.480189
5,bmi,0.283742
2,hypertension,0.125474
3,heart_disease,0.101449
0,gender_code,0.009148
4,smoking_code,0.000000


## Análise dos resultados

### **Desempenho Clínico**
A acurácia global de 41,14% reflete o desafio real do rastreamento glicêmico em populações desbalanceadas (37,7% normais vs 21% diabéticos). Contudo, o **recall de 95% para Pré-Diabetes** demonstra excelente sensibilidade para detecção do grupo de maior interesse clínico.


### **Fatores Preditivos**
**Idade (48%)** e **IMC (28%)** explicam 76% das decisões, corroborando literatura que posiciona estas variáveis como principais determinantes do risco glicêmico. A baixa importância do tabagismo (0%) sugere limitação do dataset sintético ou interação complexa com outras variáveis.

### **Limitações**
1. Dataset sintético pode não capturar variabilidade real
2. Desbalanceamento de classes afeta precisão dos diabéticos
3. Ausência de medidas antropométricas avançadas (circunferência abdominal, ABSI)

# Load (Salvando o resultado Final)

Vamos criar um arquivo CSV final que contém:

- Dados originais

- Suas classificações clínicas (Normal/Pré-DM/DM)

- Predições da IA para cada paciente

- Importância das variáveis (para relatório)

In [16]:
# --- ETAPA FINAL: LOAD (Carga) ---

# 1. Gerando predições para TODOS os pacientes (não só teste)
predicoes_todos = modelo.predict(X)
df['predicao_modelo'] = predicoes_todos
df['predicao_descricao'] = df['predicao_modelo'].map({0: 'Normal', 1: 'Pre-Diabetes', 2: 'Diabetes'})

# 2. Criando tabela final com colunas selecionadas
colunas_relatorio = [
    'gender', 'age', 'bmi', 'hypertension', 'heart_disease',
    'smoking_history', 'HbA1c_level', 'classificacao_risco',
    'predicao_descricao', 'risco_code'
]

df_final = df[colunas_relatorio].copy()

# 3. Salvando o arquivo final
df_final.to_csv('pipeline_etl_diabetes_final.csv', index=False)

# 4. Resumo final do pipeline
print("🚀 **PIPELINE ETL CONCLUÍDO COM SUCESSO!**")
print(f"📊 Pacientes processados: {len(df_final):,}")
print(f"💾 Arquivo salvo: 'pipeline_etl_diabetes_final.csv'")
print("\n📈 **RESUMO CLÍNICO FINAL:**")
print(df_final['classificacao_risco'].value_counts())
print("\n🎯 **PREDIÇÕES DO MODELO:**")
print(df_final['predicao_descricao'].value_counts())

# Mostrando as primeiras linhas do resultado final
display(df_final.head())

print("\n✅ Baixe o arquivo na aba 📁 à esquerda para usar no GitHub!")

🚀 **PIPELINE ETL CONCLUÍDO COM SUCESSO!**
📊 Pacientes processados: 100,000
💾 Arquivo salvo: 'pipeline_etl_diabetes_final.csv'

📈 **RESUMO CLÍNICO FINAL:**
classificacao_risco
Pre-Diabetes    41346
Normal          37857
Diabetes        20797
Name: count, dtype: int64

🎯 **PREDIÇÕES DO MODELO:**
predicao_descricao
Pre-Diabetes    95160
Normal           3974
Diabetes          866
Name: count, dtype: int64


,gender,age,bmi,hypertension,heart_disease,smoking_history,HbA1c_level,classificacao_risco,predicao_descricao,risco_code
0,Female,80.0,25.19,0,1,never,6.6,Diabetes,Pre-Diabetes,2
1,Female,54.0,27.32,0,0,No Info,6.6,Diabetes,Pre-Diabetes,2
2,Male,28.0,27.32,0,0,never,5.7,Pre-Diabetes,Normal,1
3,Female,36.0,23.45,0,0,current,5.0,Normal,Pre-Diabetes,0
4,Male,76.0,20.14,1,1,current,4.8,Normal,Pre-Diabetes,0



✅ Baixe o arquivo na aba 📁 à esquerda para usar no GitHub!
